In [1]:
import torch

from ptgnn.transform.ptree_matrix import permutation_tree_to_matrix

In [2]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [3]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'transformation_mode': 'chienn_tree_basic',
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [4]:
train_loader, _, val_loader = fetch_loaders(data_config=data_config)

In [5]:
for batch in val_loader:
    display(batch)
    break

DataBatch(x=[2356, 118], edge_index=[2, 6416], edge_attr=[6416, 80], pos=[2356, 6], y=[32], batch=[2356], ptr=[33], ptree=[2356])

# part in collation and part possible in preprocessing

In [6]:
k = 3

In [7]:
# pt = [batch.ptree[0]]
pt = ['{"P": [1, {"Z": [3, 4, 5]}]}']
k = 2
display(pt)
idx_matrix, type_matrix = permutation_tree_to_matrix(batch.ptree, k)
display(idx_matrix)
display(type_matrix)

['{"P": [1, {"Z": [3, 4, 5]}]}']

tensor([[   0,    0,    1,    1],
        [   1,    0,    0,    0],
        [   1,    1,    0,    3],
        ...,
        [2354,    1,    1, 2351],
        [2354,    1,    2, 2353],
        [2355,    0, 2354, 2354]])

tensor([[1, 0],
        [1, 0],
        [1, 2],
        ...,
        [1, 2],
        [1, 2],
        [1, 0]])

In [11]:
from ptgnn.transform.ptree_matrix import permutation_tree_to_order_matrix
b = permutation_tree_to_order_matrix(batch, 1)
display(b)
display(b.initial_map)
display(b.layer0_order_matrix[0][0][:, -10:])
display(b.layer0_type_mask[0][0][-10:])

DataBatch(x=[2356, 118], edge_index=[2, 6416], edge_attr=[6416, 80], pos=[2356, 6], y=[32], batch=[2356], ptr=[33], ptree=[2356], initial_map=[6416], layer0_order_matrix=[1], layer0_type_mask=[1], layer0_pooling=[1], num_layer=2, layer1_order_matrix=[1], layer1_type_mask=[1], layer1_pooling=[1])

tensor([   1,    0,    3,  ..., 2351, 2353, 2354])

tensor([[6406, 6407, 6408, 6409, 6410, 6411, 6412, 6413, 6414, 6415]],
       dtype=torch.int32)

tensor([0, 2, 2, 2, 0, 0, 2, 2, 2, 0])

## From here second part that is either in the model or in preprocessing

In [16]:
# load first index range of elements
data_array = batch.x[idx_matrix[:, -1]]

In [17]:
# get structure to orient to
idx_structure = idx_matrix[:, :-1]

### for loop starts here

In [18]:
# get indexes for graph pooling
idx_structure, current_layer_pooling_counts = torch.unique(idx_structure[:, :-1], dim=0, return_counts=True)
display(idx_structure)
display(current_layer_pooling_counts)

tensor([[   0,    0],
        [   1,    0],
        [   1,    1],
        ...,
        [2354,    0],
        [2354,    1],
        [2355,    0]])

tensor([1, 1, 3,  ..., 1, 3, 1])

In [19]:
# get indexes for graph pooling
current_layer_pooling = torch.repeat_interleave(current_layer_pooling_counts)
display(current_layer_pooling[:10])

tensor([0, 1, 2, 2, 2, 3, 4, 4, 4, 5])

In [12]:
# init circling
# todo: rework for other types
order_matrix = torch.zeros(k, len(current_layer_pooling), dtype=torch.int) - 1

In [13]:
cur_pos = 0
for i in current_layer_pooling_counts:#[:5]:
    current_k = min(k, i)
    r = torch.arange(cur_pos, cur_pos+i)
    for j in range(current_k):
        order_matrix[:current_k, cur_pos+j] = torch.roll(r, shifts=-j)
    cur_pos += i

In [14]:
order_matrix

tensor([[   0,    1,    2,  ..., 6413, 6414, 6415],
        [  -1,   -1,    3,  ..., 6414, 6412,   -1],
        [  -1,   -1,    4,  ..., 6412, 6413,   -1]], dtype=torch.int32)

In [15]:
# add zero padding to data list
data_array = torch.cat([torch.zeros(1, data_array.shape[-1]), data_array], dim=0)
order_matrix += 1

In [16]:
temp = data_array[order_matrix]

In [17]:
temp.shape

torch.Size([3, 6416, 118])

In [18]:
# apply z layer

In [19]:
mask_z = type_matrix[:, -1] == 2

In [23]:
z_layer = torch.nn.ModuleList([
    torch.nn.Linear(118, 118)
    for _ in range(k)
])
z_final_layer = torch.nn.Linear(118, 118)
z_elu = torch.nn.ELU()

In [21]:
# todo: take care of duplicate elements that are sent through linear layer

In [22]:
data_array.shape

torch.Size([6417, 118])

In [23]:
embedding = torch.stack([
    emb(t)
    for emb, t in zip(z_layer,temp)
], dim=1)
display(embedding.shape)

torch.Size([6416, 3, 118])

In [24]:
temp3 = embedding.sum(dim=1)
display(temp3.shape)

torch.Size([6416, 118])

In [25]:
temp4 = z_final_layer(temp3)
display(temp4.shape)

torch.Size([6416, 118])

In [7]:
import torch_geometric
data_array = torch_geometric.nn.global_add_pool(temp4, current_layer_pooling)

NameError: name 'temp4' is not defined

In [27]:
data_array.shape

torch.Size([4124, 118])

In [20]:
k = 3

In [21]:
batch.idx_matrix, batch.type_matrix = permutation_tree_to_matrix(batch.ptree, 3)

In [24]:
device = 'cpu'
batch = batch.to(device)
z_elu = z_elu.to(device)
z_layer = z_layer.to(device)
z_final_layer = z_final_layer.to(device)

In [25]:
# make link to batch idx_matrix
idx_matrix = batch.idx_matrix

# load first index range of elements
data_array = batch.x[idx_matrix[:, -1]]

# get structure to orient to
idx_structure = idx_matrix[:, :-1]

In [26]:
# get indexes for graph pooling
idx_structure, current_layer_pooling_counts = torch.unique(idx_structure[:, :-1], dim=0, return_counts=True)

# get indexes for graph pooling
current_layer_pooling = torch.repeat_interleave(current_layer_pooling_counts)

# init circling
# todo: rework for other types - treat everything as Z
order_matrix = torch.zeros(k, len(current_layer_pooling), dtype=torch.int) - 1

In [27]:
cur_pos = 0
for i in current_layer_pooling_counts:
    current_k = min(k, i)
    r = torch.arange(cur_pos, cur_pos+current_k)
    for j in range(i):
        order_matrix[:current_k, cur_pos+j] = torch.roll(r, shifts=-j)
    cur_pos += i

In [28]:
order_matrix[:, :10]

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [-1, -1,  3,  4,  2, -1,  7,  8,  6, -1],
        [-1, -1,  4,  2,  3, -1,  8,  6,  7, -1]], dtype=torch.int32)

In [29]:
# add zero padding to data list
data_array = torch.cat([torch.zeros(1, data_array.shape[-1], device=data_array.device), data_array], dim=0)
order_matrix += 1

embedding = data_array[order_matrix]
# mask_z = batch.type_matrix

embedding = torch.stack([
    emb(t)
    for emb, t in zip(z_layer, embedding)
], dim=1)

embedding[(order_matrix.T == 0).unsqueeze(-1).expand_as(embedding)] = 0.

embedding = embedding.sum(dim=1)
embedding = z_final_layer(embedding)

# global pooling
data_array = torch_geometric.nn.global_add_pool(embedding, current_layer_pooling)

NameError: name 'torch_geometric' is not defined

In [9]:
batch

DataBatch(x=[2104, 118], edge_index=[2, 5960], edge_attr=[5960, 80], pos=[2104, 6], y=[28], batch=[2104], ptr=[29], ptree=[2104], num_layer=[1], initial_map=[5960], layer0_order_matrix=[3, 5960], layer0_type_mask=[5960], layer0_pooling=[5960], layer1_order_matrix=[3, 3736], layer1_type_mask=[3736], layer1_pooling=[3736])

In [10]:
import json
ptree = json.loads(batch.ptree[0])

In [11]:
from ptgnn.transform.ptree_matrix import permutation_tree_depth

# get depth of tree
tree_depth = permutation_tree_depth(ptree)

In [12]:
batch.ptree[1]

'{"P": [0, {"Z": [7, 9]}]}'

In [14]:
permutation_tree_to_matrix([batch.ptree[1]])

NameError: name 'permutation_tree_to_matrix' is not defined

In [1]:
from ptgnn.transform.ptree_matrix import permutation_tree_to_order_matrix

permutation_tree_to_order_matrix(batch, 3)

NameError: name 'batch' is not defined

In [1]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [2]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'transformation_mode': 'chienn_tree_order_matrix',
        'transformation_parameters': {
            'k': 3
        }
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [3]:
train_loader, _, val_loader = fetch_loaders(data_config=data_config)

In [4]:
for batch in train_loader:
    display(batch)
    break

DataBatch(x=[2804, 118], edge_index=[2, 7684], edge_attr=[7684, 80], pos=[2804, 6], y=[32], batch=[2804], ptr=[33], ptree=[2804], num_layer=[1], initial_map=[7684], layer0_order_matrix=[3, 7684], layer0_type_mask=[7684], layer0_pooling=[7684], layer1_order_matrix=[3, 4978], layer1_type_mask=[4978], layer1_pooling=[4978])

In [5]:
from ptgnn.model.modules.ptree.complex_ptree_layer import ComplexPtreeLayer
from tqdm import tqdm
k=3
device = 'cuda'  # 'cpu'
model = ComplexPtreeLayer(k, 118).to(device)

In [10]:
for batch in tqdm(train_loader):
    batch = batch.to(device)
    model(batch)

100%|██████████| 1721/1721 [00:54<00:00, 31.62it/s]


In [5]:
print(batch.layer1_order_matrix.max())
print(batch.layer0_pooling.max())
print(batch.layer0_type_mask.max())
print(batch.initial_map.max())

tensor(4977, dtype=torch.int32)
tensor(4977)
tensor(2)
tensor(2803)


In [21]:
batch.layer1_order_matrix[:, :10]

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], device='cuda:0',
       dtype=torch.int32)

In [22]:
batch.layer1_pooling[:10]

tensor([0, 1, 1, 2, 2, 3, 3, 4, 4, 5], device='cuda:0')

In [5]:
k=3

In [6]:
import torch
import torch_geometric
from tqdm import tqdm

In [7]:
z_layer = torch.nn.ModuleList([
    torch.nn.Linear(118, 118)
    for _ in range(k)
])
z_final_layer = torch.nn.Linear(118, 118)
z_elu = torch.nn.ELU()

In [8]:
# device = 'cpu'
device = 'cuda'

z_elu = z_elu.to(device)
z_layer = z_layer.to(device)
z_final_layer = z_final_layer.to(device)

In [4]:
for batch in tqdm(train_loader):
    batch = batch.to(device)

    # load first index range of elements
    data_array = batch.x[batch.initial_map]

    for layer_idx in range(batch.num_layer):
        # fetch data
        order_matrix = batch[f"layer{layer_idx}_order_matrix"]
        current_layer_pooling = batch[f"layer{layer_idx}_pooling"]
        type_mask = batch[f"layer{layer_idx}_type_mask"] # todo: actually use for different types

        # add zero padding to data list
        data_array = torch.cat([torch.zeros(1, data_array.shape[-1], device=data_array.device), data_array], dim=0)
        order_matrix += 1

        embedding = data_array[order_matrix]
        # mask_z = batch.type_matrix

        embedding = torch.stack([
            emb(t)
            for emb, t in zip(z_layer, embedding)
        ], dim=1)

        embedding[(order_matrix.T == 0).unsqueeze(-1).expand_as(embedding)] = 0.

        embedding = embedding.sum(dim=1)
        embedding = z_final_layer(embedding)

        # global pooling
        data_array = torch_geometric.nn.global_add_pool(embedding, current_layer_pooling)

    assert data_array.shape == batch.x.shape

NameError: name 'tqdm' is not defined

In [9]:
batch = batch.to(device)

# load first index range of elements
data_array = batch.x[batch.initial_map]

In [10]:
layer_idx = 0

In [11]:
# fetch instructions for this layer
order_matrix = batch[f"layer{layer_idx}_order_matrix"]
current_layer_pooling = batch[f"layer{layer_idx}_pooling"]
type_mask = batch[f"layer{layer_idx}_type_mask"]

In [12]:
# order matrix contains -1. This is due to not all permutations requiring k elements to connect.
# solve this with creating new index zero that is full of zeros and increasing order_matrix index
data_array = torch.cat(
    [
        torch.zeros(1, data_array.shape[-1], device=data_array.device),
        data_array
    ],
    dim=0
)
order_matrix += 1

In [13]:
# create order mask (required later to remove elements that were -1 -> 0 but after linear layer they
# are no longer 0)
mask_order_matrix = order_matrix == 0

In [14]:
# change indices from order list to real data (technically this inflates the matrix, but all are references
# so no actual space should be required.
data_array = data_array[order_matrix]

In [15]:
# do operation for each layer and put result into a temporary array
# array initialized with the first column to cover the case of doing nothing (tree extension for uniform
# layer count)
temporary_data_array = data_array[0].clone()

In [16]:
# for loop
node_value = 2

In [17]:
# create mask for this type:
# - create mask
# - extend the mask to match
mask = type_mask == node_value
display(mask)
display(mask.shape)

tensor([False, False,  True,  ...,  True, False,  True], device='cuda:0')

torch.Size([7684])

In [18]:
# iterate over k and get embeddings
temp = torch.stack([
    emb(t)
    for emb, t in zip(z_layer, data_array[:, mask])
], dim=0)
# set entries that were previously only the -1's to zero
temp[mask_order_matrix[:, mask]] = 0.
# sum k (shifted [in order_matrix] embeddings)
temp = temp.sum(dim=0)
# run through final layer
temp = z_final_layer(temp)
# put into temporary_array
temporary_data_array[mask] = temp

In [19]:
# end of for loop

In [20]:
# global pooling
data_array = torch_geometric.nn.global_add_pool(temporary_data_array, current_layer_pooling)

In [21]:
data_array = z_elu(data_array)

In [22]:
type_mask2 = type_mask[torch.unique(current_layer_pooling)]

# Convert trees from C to Z, from Q to S

In [12]:
permutation_trees = batch.ptree
display(permutation_trees)

['{"P": [1, {"Z": [3, 5]}]}',
 '{"P": [0, {"Z": [7, 9]}]}',
 '{"P": [3, {"Z": [1, 5]}]}',
 '{"P": [2, {"Z": [14, 93]}]}',
 '{"P": [5, {"Z": [1, 3]}]}',
 '{"P": [4]}',
 '{"P": [7, {"Z": [0, 9]}]}',
 '{"P": [6, {"Z": [11]}]}',
 '{"P": [9, {"Z": [0, 7]}]}',
 '{"P": [8]}',
 '{"P": [11, {"Z": [6]}]}',
 '{"P": [10, {"Z": [13, 15]}]}',
 '{"P": [13, {"Z": [10, 15]}]}',
 '{"P": [12, {"Z": [17, 19, 21]}]}',
 '{"P": [15, {"Z": [10, 13]}]}',
 '{"P": [14, {"Z": [2, 93]}]}',
 '{"P": [17, {"Z": [12, 21, 19]}]}',
 '{"P": [16, {"Z": [23, 25]}]}',
 '{"P": [19, {"Z": [12, 17, 21]}]}',
 '{"P": [18]}',
 '{"P": [21, {"Z": [12, 19, 17]}]}',
 '{"P": [20]}',
 '{"P": [23, {"Z": [16, 25]}]}',
 '{"P": [22, {"Z": [27, 31, 29]}]}',
 '{"P": [25, {"Z": [16, 23]}]}',
 '{"P": [24, {"Z": [46, 89, 91]}]}',
 '{"P": [27, {"Z": [22, 29, 31]}]}',
 '{"P": [26, {"Z": [33, 35, 37]}]}',
 '{"P": [29, {"Z": [22, 31, 27]}]}',
 '{"P": [28]}',
 '{"P": [31, {"Z": [22, 27, 29]}]}',
 '{"P": [30]}',
 '{"P": [33, {"Z": [26, 37, 35]}]}',
 

In [13]:
ptree = permutation_trees[0]

In [14]:
import json
display(json.loads(ptree))

{'P': [1, {'Z': [3, 5]}]}

In [15]:
ptree = {
    'Q': [
        1,
        {
            'C': [
                2,
                3,
                4,
                5,
            ]
        }
    ]
}

In [16]:
def invert_ptree(perm_tree: dict):
    key = next(iter(perm_tree.keys()))
    return {
        key: perm_tree[key][::-1]
    }

In [23]:
def remove_inverted_through_p(perm_tree: dict):
    if isinstance(perm_tree, int):
        return perm_tree

    key = next(iter(perm_tree.keys()))
    if key == "Q":
        return {
            "P": [
                {
                    "S": [
                        remove_inverted_through_p(subtree)
                        for subtree in perm_tree[key]
                    ]
                },
                {
                    "S": [
                        remove_inverted_through_p(subtree)
                        for subtree in perm_tree[key]
                    ][::-1]
                }
            ]
        }
    elif key == "C":
        return {
            "P": [
                {
                    "Z": perm_tree[key]
                },
                {
                    "Z": [
                             remove_inverted_through_p(subtree)
                             for subtree in perm_tree[key]
                         ][::-1]
                }
            ]
        }
    else:
        return perm_tree

In [26]:
remove_inverted_through_p(ptree)

{'P': [{'S': [1, {'P': [{'Z': [2, 3, 4, 5]}, {'Z': [5, 4, 3, 2]}]}]},
  {'S': [{'P': [{'Z': [2, 3, 4, 5]}, {'Z': [5, 4, 3, 2]}]}, 1]}]}

In [ ]:
transformed = {
    'P': [
        {
            'S': [
                1,
                {
                    'P': [
                        {
                            'Z': [2, 3, 4, 5]
                        },
                        {
                            'Z': [5, 4, 3, 2]
                        }
                    ]
                }
            ]
        },
        {
            'S': [
                {
                    'P': [
                        {
                            'Z': [2, 3, 4, 5]
                        },
                        {
                            'Z': [5, 4, 3, 2]
                        }
                    ]
                },
                1
            ]
        }
    ]
}

## Testing dataset config for permutation tree generation

In [1]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [2]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'graph_mode': 'vertex',
        'transformation_mode': 'permutation_tree',
        'transformation_parameters': {
            'k': 3
        }
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [3]:
train_loader, _, val_loader = fetch_loaders(data_config=data_config)

In [4]:
for batch in val_loader:
    # display(batch)
    # break
    continue